# Keyword Spotting (KWS) Model Training

This notebook demonstrates how to train a Keyword Spotting model using TensorFlo

In [ ]:
# Install required packages
!pip install numpy>=1.19.2 librosa>=0.8.0 scipy>=1.6.0 scikit-learn>=0.24.0 \
    soundfile>=0.10.3 matplotlib>=3.3.0 tensorflow>=2.8.0 sounddevice>=0.4.4 \
    pandas>=1.3.0 tqdm>=4.62.0 kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your kaggle.json file

# Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import os
import zipfile
import shutil
from pathlib import Path

# Create necessary directories
os.makedirs('data', exist_ok=True)
os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)

# Download dataset from Kaggle
!kaggle competitions download -c tensorflow-speech-recognition-challenge -p data

# Install 7zip for extraction
!apt-get install p7zip-full

# Extract dataset
!7z x data/train.7z -odata/temp

# Define the classes we want to use (10 specific commands)
target_classes = [
    'yes', 'no', 'up', 'down', 'left', 'right',
    'on', 'off', 'stop', 'go'
]

# Organize data into train/val splits
def organize_data(source_dir, train_dir, val_dir, val_split=0.2):
    total_files = 0
    for class_name in target_classes:
        class_dir = os.path.join(source_dir, class_name)
        if os.path.isdir(class_dir):
            # Create class directories in train and val
            os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
            
            # Get all files for this class
            files = [f for f in os.listdir(class_dir) if f.endswith('.wav')]
            total_files += len(files)
            
            # Split into train and validation
            val_size = int(len(files) * val_split)
            val_files = files[:val_size]
            train_files = files[val_size:]
            
            # Move files to respective directories
            for f in train_files:
                shutil.copy(os.path.join(class_dir, f), 
                           os.path.join(train_dir, class_name, f))
            for f in val_files:
                shutil.copy(os.path.join(class_dir, f), 
                           os.path.join(val_dir, class_name, f))
            
            print(f"Processed {class_name}: {len(train_files)} train, {len(val_files)} validation files")
    
    print(f"\nTotal files processed: {total_files}")

# Organize the data
organize_data('data/temp/train/audio', 'data/train', 'data/val')

# Clean up temporary directory
shutil.rmtree('data/temp')

print("\nDataset preparation completed!")

In [ ]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

# Add the src directory to Python path
sys.path.append('src')

from model import KWSSystem
from feature_extraction import FeatureExtractor
from data_loader import AudioDataLoader

In [ ]:
# Training configuration
config = {
    'data_dir': 'data',
    'model_type': 'cnn',  # or 'lstm'
    'feature_type': 'mfcc',  # or 'mel'
    'batch_size': 32,
    'epochs': 50,
    'learning_rate': 0.001,
    'num_classes': len(target_classes)  # Now 10 classes
}

print(f"Training configuration:")
print(f"- Model type: {config['model_type']}")
print(f"- Feature type: {config['feature_type']}")
print(f"- Number of classes: {config['num_classes']}")
print(f"- Batch size: {config['batch_size']}")
print(f"- Epochs: {config['epochs']}")
print(f"- Learning rate: {config['learning_rate']}")

# Create feature extractor
feature_extractor = FeatureExtractor()

# Initialize KWS system
kws = KWSSystem(
    model_type=config['model_type'],
    n_mfcc=13 if config['feature_type'] == 'mfcc' else None,
    num_classes=config['num_classes']
)

In [ ]:
def create_dataset(data_dir, feature_extractor, batch_size